<a href="https://colab.research.google.com/github/Lamblador/IR_expert_system/blob/main/Koshelev_IR_expert_system_2_First_model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IR spectral analysis of organic and coordination compounds via machine learning and neural networks
By: Koshelev Daniil 

Based on some code form: https://github.com/rayguna/IR-Spectra-Analyzer
Предыдущая часть: https://github.com/Lamblador/IR_expert_system/blob/main/Koshelev_IR_expert_system_1_Dataset_Preparation.ipynb

-Work in progress (last updated:07/03/2023)-

B. Цель данного блокнота

Целью этого блокнота является обучение примитивной модели распознавания всего лишь 15 классов (функциональных групп) в спектре из 1800 точек в диапазоне 500-4100 [1/cm] 

C. План реализации

1. Импортировать табличные данные разметки классов по CAS и табличные данные уже обработанных спектров нужного диапазона.
2. Создать объект класса Dataset
3. Реализовать код для обучения модели.
4. Реализовать саму модель.
5. Обучить модель.
6. Оценить точность модели в мультиклассовом анализе.

NOTES
1. Импортированы табличные данные.

Клонирование данных с GitHub. На сайте выложены как исходные списки веществ, так и скачанные спектры соединений, которые были запарсены кодом ниже.

In [1]:
!git clone https://github.com/Lamblador/IR_expert_system.git

Cloning into 'IR_expert_system'...
remote: Enumerating objects: 16750, done.
remote: Counting objects: 100% (1907/1907), done.
remote: Compressing objects: 100% (1853/1853), done.
remote: Total 16750 (delta 63), reused 1894 (delta 53), pack-reused 14843
Receiving objects: 100% (16750/16750), 244.28 MiB | 10.77 MiB/s, done.
Resolving deltas: 100% (423/423), done.
Updating files: 100% (16623/16623), done.


In [2]:
# Установка нужных пакетов
from IPython.core.display import clear_output

clear_output()

In [3]:
# Вводятся нужные импорты
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
import glob
import os
import numpy as np
import torch
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import datasets

# Импорт табличных данных

In [4]:
labels_df = pd.read_csv('/content/IR_expert_system/data/nist_ir_spectra_inchi_group_list.csv')
labels_df = labels_df.set_index("CAS")
labels_df

,Name,Formula,Mw,Elements,InChi,SMILES,alkane,methyl,alkene,alkyne,...,aromatics,alkyl halides,esters,ketones,aldehydes,carboxylic acids,ether,acyl halides,amides,nitro
CAS,,,,,,,,,,,,,,,,,,,,,
100-00-5,"Benzene, 1-chloro-4-nitro-",C6H4ClNO2,156.993056,"['C', 'H', 'Cl', 'N', 'O']",InChI=1/C6H4ClNO2/c7-5-1-3-6(4-2-5)8(9)10/h1-4H,[O-][N+](=O)c1ccc(Cl)cc1,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
100-01-6,p-Nitroaniline,C6H6N2O2,138.042927,"['C', 'H', 'N', 'O']",InChI=1/C6H6N2O2/c7-5-1-3-6(4-2-5)8(9)10/h1-4H...,Nc1ccc(cc1)[N+]([O-])=O,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
100-02-7,"Phenol, 4-nitro-",C6H5NO3,139.026943,"['C', 'H', 'N', 'O']","InChI=1/C6H5NO3/c8-6-3-1-5(2-4-6)7(9)10/h1-4,8H",Oc1ccc(cc1)[N+]([O-])=O,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1000-49-3,N-butyltrimethylsilane,C7H18Si,130.117777,"['C', 'H', 'Si']","InChI=1/C7H18Si/c1-5-6-7-8(2,3)4/h5-7H2,1-4H3",CCCC[Si](C)(C)C,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1000-50-6,N-Butyldimethylchlorosilane,C6H15ClSi,150.063155,"['C', 'H', 'Cl', 'Si']","InChI=1/C6H15ClSi/c1-4-5-6-8(2,3)7/h4-6H2,1-3H3",CCCC[Si](C)(C)Cl,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99-97-8,"Benzenamine, N,N,4-trimethyl-",C9H13N,135.104799,"['C', 'H', 'N']","InChI=1/C9H13N/c1-8-4-6-9(7-5-8)10(2)3/h4-7H,1...",CN(C)c1ccc(C)cc1,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999-78-0,"2-Pentyne, 4,4-dimethyl-",C7H12,96.093900,"['C', 'H']","InChI=1/C7H12/c1-5-6-7(2,3)4/h1-4H3",CC#CC(C)(C)C,1.0,1.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
999-81-5,"Ammonium chloride, 2-chloroethyltrimethyl-",C5H13ClN.Cl,157.042505,"['C', 'H', 'Cl', 'N', 'Cl']","InChI=1/C5H13ClN.ClH/c1-7(2,3)5-4-6;/h4-5H2,1-...",[Cl-].C[N+](C)(C)CCCl,1.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
data_df = pd.read_csv('/content/IR_expert_system/data/ir_spectra_padded_inter_full.csv')
data_df = data_df.set_index('cas registry no')
data_df

,Unnamed: 0,molform,x,x1,y,y1
cas registry no,,,,,,
484-17-3,0,C 14 H 10 O,[ 550. 554. 558. 562. 566. 570. 574. 57...,[ 500 502 504 ... 4094 4096 4098],[1.56400e-04 2.53300e-04 3.04300e-04 4.55600e-...,[0.0001564 0.0001564 0.0001564 ... 0.0001938 0...
892-75-1,1,C17 H30 Cl N O,[ 667. 668. 669. ... 3998. 3999. 4000.],[ 500 502 504 ... 4094 4096 4098],[0.02079999 0.02109998 0.02179998 ... 0.457799...,[0.02079999 0.02079999 0.02079999 ... 0.457799...
3208-26-2,2,C 15 H 24 O,[ 549.759 551.688 553.617 ... 3996.842 3998....,[ 500 502 504 ... 4094 4096 4098],[0.0018277 0.0009145 0.0013071 ... 0.0012858 0...,[0.0018277 0.0018277 0.0018277 ... 0.00236 0...
29006-25-5,3,C28 H55 N O4,[ 574.38 575.50696 576.634 ... 3985.735...,[ 500 502 504 ... 4094 4096 4098],[0.677 0.677 0.677 ... 0.069000...,[0.67699999 0.67699999 0.67699999 ... 0.069000...
38411-17-5,4,C 8 H 6 Cl 2 N 2 O 3,[ 549.759 551.688 553.617 ... 3996.842 3998....,[ 500 502 504 ... 4094 4096 4098],[0.0124 0.01257 0.0116 ... 0.000472 0...,[0.0124 0.0124 0.0124 ... 0.0010095 0...
...,...,...,...,...,...,...
116400-95-4,14341,C14 H11 Cl N2 S2,[ 399. 400. 401. ... 3796. 3797. 3798.],[ 500 502 504 ... 4094 4096 4098],[0.4749 0.47579998 0.47719997 ... 0.7722 ...,[0.56379998 0.6146 0.60969996 ... 0.772499...
4609-87-4,14342,C10 H21 N O2,[ 629.5406 629.80426 630.0681 ... 4918.083...,[ 500 502 504 ... 4094 4096 4098],[0.385 0.385 0.385 ... 0.089999...,[0.38499999 0.38499999 0.38499999 ... 0.103999...
116529-37-4,14343,C12 H26 Cl N3 O S,[ 251.243 252.7733 254.30359 ... 3793.883...,[ 500 502 504 ... 4094 4096 4098],[0.708 0.708 0.7 ... 0.138000...,[0.27600002 0.27600002 0.27600002 ... 0.138000...


In [8]:
data_x1_df = pd.read_pickle('/content/IR_expert_system/data/ir_spectra_x1_zip.csv', compression = 'zip')

In [10]:
data_y1_df = pd.read_pickle('/content/IR_expert_system/data/ir_spectra_y1_zip.zip', compression = 'zip')

BadZipFile: ignored

In [ ]:
data_df['y1'][0]

'[0.0001564 0.0001564 0.0001564 ... 0.0001938 0.0001938 0.0001938]'

In [ ]:
dataset_df = labels_df.join(data_df)
x = dataset_df.iloc[:,-1][0]
x = x[1:-2]
x

'0.02607692 0.02797573 0.03253286 ... 0.00708887 0.00708887 0.0070888'

In [ ]:
x = x.split()
x = [float(i) for i in x]
x

ValueError: ignored

# Создание класса Dataset

Вивлеты - преобразование для детектирования пиков.

In [ ]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset
from torchvision import datasets

class IRSpectraDataset(Dataset):
    def __init__(self):
        self.dataset_df = self.data_load_form_csv()
        self.spectra_labels = dataset_df.iloc[:,6:23]
        self.spectra_x = torch.Tensor(dataset_df.iloc[:,-3])
        self.spectra_y = torch.Tensor(dataset_df.iloc[:,-1])
        self.spectra =  torch.cat(self.spectra_x, self.spectra_y)

    def data_load_form_csv(self):
      labels_df = pd.read_csv('/content/IR_expert_system/data/nist_ir_spectra_inchi_group_list.csv')
      labels_df = labels_df.set_index("CAS")
      data_df = pd.read_csv('/content/IR_expert_system/data/ir_spectra_padded_inter_full.csv')
      data_df = data_df.set_index('cas registry no')
      dataset_df = labels_df.join(data_df)
      return dataset_df


    def __len__(self):
        return len(self.dataset_df)

    def __getitem__(self, idx):
        spectra = self.spectra[idx]
        label = self.spectra_labels[idx]
        #transform
        return spectra, label

In [ ]:
IRSpectraDataset = IRSpectraDataset()
IRSpectraDataset.spectra

ValueError: ignored

# Код для обучения

Взят из https://github.com/Gan4x4/ml_snippets/blob/main/Training/Trainer.ipynb

In [ ]:
import torch
from tqdm import tqdm

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def get_correct_count(pred, labels):
    _, predicted = torch.max(pred.data, 1)
    return (predicted.cpu() == labels.cpu()).sum().item()


@torch.inference_mode()  # this annotation disable grad computation
def validate(model, test_loader, device="cpu"):
    correct, total = 0, 0
    for imgs, labels in test_loader:
        pred = model(imgs.to(device))
        total += labels.size(0)
        correct += get_correct_count(pred, labels)
    return correct / total

In [ ]:
class Trainer:
    def __init__(self, model, plotter=None, lr=0.03):
        self.model = model
        self.optimizer = torch.optim.SGD(
            self.model.parameters(), lr=0.03
        )  # Weight update
        self.criterion = nn.CrossEntropyLoss()  # Loss function
        self.plotter = ProgressPlotter() if plotter is None else plotter
        self.epochs = 25
        self.loss_hist = []

    def __call__(self, train_loader, val_loader, epochs=10):
        global device
        print("Using device:", device)
        self.model.to(device)
        self.model.train()
        for epoch in tqdm(range(self.epochs)):
            self.loss_hist = []
            correct, total = 0, 0
            for imgs, labels in train_loader:
                correct += self.process_batch(imgs, labels)
                total += len(labels)
            self.plotter.add_scalar("Loss/train", np.mean(self.loss_hist))
            self.plotter.add_scalar(
                "Accuracy/val", validate(self.model, val_loader, device=device)
            )
            self.plotter.add_scalar("Accuracy/train", correct / total)
            self.plotter.display(["Loss/train", "Accuracy/val"])

    def process_batch(self, imgs, labels):
        self.optimizer.zero_grad()
        out = self.model(imgs.to(device))
        loss = self.criterion(out, labels.to(device))
        loss.backward()
        self.loss_hist.append(loss.item())
        self.optimizer.step()
        return get_correct_count(out.cpu(), labels)

Код для визуализации из прошлых заданий, можно заменить его собственым.

In [ ]:
from IPython.display import clear_output
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np


class ProgressPlotter:
    def __init__(self, title="default", groups=None) -> None:
        self._history_dict = defaultdict(dict)
        self.set_title(title)
        self.groups = self.get_groups(groups)

    def get_groups(self, groups):
        if groups is not None:
            return self._history_dict.keys()
        if type(groups) is str:
            groups = [groups]
        return groups

    def set_title(self, title):
        for g in self._history_dict.keys():
            self._history_dict[g][title] = []  # reset data
        self.title = title

    # group e.g. "loss_val" tag e.g. "experiment_1"
    def add_scalar(self, group: str, value, tag=None) -> None:
        tag = self.title if tag is None else tag

        if not tag in self._history_dict[group]:
            self._history_dict[group][tag] = []
        self._history_dict[group][tag].append(value)

    def add_row(self, group: str, value, tag=None) -> None:
        tag = self.title if tag is None else tag
        self._history_dict[group][tag] = value

    def display_keys(self, ax, data):
        history_len = 0
        ax.grid()
        for key in data:
            ax.plot(data[key], label=key)
            history_len = max(history_len, len(data[key]))
        if len(data) > 1:
            ax.legend(loc="upper right")
        if history_len < 50:
            ax.set_xlabel("step")
            ax.set_xticks(np.arange(history_len))
            ax.set_xticklabels(np.arange(history_len))

    """
     groups list of keys like [['loss_train','loss_val'],['accuracy']]
     All charts within a group will be plot in the same axis
  """

    def display(self, groups=None):
        clear_output()
        if groups is None:
            groups = self.groups
        n_groups = len(groups)
        fig, ax = plt.subplots(1, n_groups, figsize=(48 // n_groups, 3))
        if n_groups == 1:
            ax = [ax]
        for i, g in enumerate(groups):
            ax[i].set_ylabel(g)
            self.display_keys(ax[i], self.history_dict[g])
        fig.tight_layout()
        plt.show()

    @property
    def history_dict(self):
        return dict(self._history_dict)

# Модель

Предлагается использовать модель с несколькими свёрточными слоями(1) и понижением размерности до 225-450, а затем с полносвязыными слоями, где на выходе одна из 17 меток классов.

In [ ]:
from torch.nn.modules.pooling import MaxPool2d
import torch.nn as nn
import torch

class TwoLayerCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv_stack = nn.Sequential(
            nn.Conv1d(2, 8, 3), # in channel=2, out=32 3 - ядро свертки, таблица 1800 точек*2 столбца
            nn.MaxPool1d(2), # size [34,1800] -> [34,900] 
            nn.ReLU(),
            nn.Conv1d(8, 16, 3), # in channel=34, out=34
            nn.MaxPool1d(2), #  size [34,900] -> [34,450] 
            nn.ReLU(),
            nn.Flatten(),
            nn.Linear(7*7*32, 100),
            nn.ReLU(),
            nn.Linear(100, 10)
            )

    def forward(self, x):
        # Your code here
        x = self.conv_stack(x) 
        return x

# Обучение модели

Оптимизатор - Адам с базовыми настройками
Лосс - Бинарная кросс-энтропия
Вывести табличку по классам sklearn пресижен реколл по классам
Какой-то репорт в sklearn 


# Оценка качества модели

# Выводы